In [1]:
data_var = '2024-01-18'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10547,2024-01-18,Eua Nba,21:30,New York Knicks,Washington Wizards,1.16,5.77,233.5,1.85,1.97,-12.5,2.02,1.83,8pGNrn1h,0.862069,0.173310,0.540541,0.507614,0.035379,165.048,46.053279,0.279030,1.8,1.643168,0.912871,172.22,1.500,0.288704,0.192469,47.0,394.462,138.644664,0.351478,0.6,1.341641,2.236068,173.16,3.386,1.078810,0.318609,-16.0,109,117,1.58,1.48,163.784,499.908,0.940768,0.044426,0.069792,1.14,0.228,0.701754,0.668449,0.8,0.131551,-1.07,-0.214,-22.289720,0.212586,0.2,-0.012586
10548,2024-01-18,Mundo Liga Dos Campeões Das Américas,19:10,Boca Juniors,Flamengo,2.68,1.43,153.5,1.83,1.87,5.5,1.82,1.88,M9VR9cOI,0.373134,0.699301,0.546448,0.534759,0.072435,121.418,13.616458,0.112145,1.8,1.643168,0.912871,122.24,1.442,0.291839,0.202385,33.0,103.260,19.560978,0.189434,1.8,1.643168,0.912871,101.92,1.246,0.284130,0.228034,6.0,64,91,1.91,1.12,108.498,108.326,0.430114,0.015289,0.022933,-2.48,-0.496,-3.387097,0.000000,0.0,0.000000,-1.51,-0.302,-1.423841,0.793396,0.7,-0.093396
10549,2024-01-18,Eua Ncaa,20:07,Binghamton,Vermont,3.48,1.33,140.5,1.80,1.95,5.5,2.10,1.67,nFReqA26,0.287356,0.751880,0.555556,0.512821,0.039236,246.660,203.168413,0.823678,1.2,1.643168,1.369306,189.00,3.570,3.333534,0.933763,-37.0,87.678,13.291483,0.151594,2.4,1.341641,0.559017,80.56,1.230,0.208087,0.169176,33.0,75,76,2.52,1.06,0.000,152.246,0.632133,0.056569,0.161303,0.00,0.000,inf,0.000000,0.0,0.000000,0.98,0.196,1.683673,0.000000,0.0,0.000000
10550,2024-01-18,Eua Ncaa,21:00,Albany,UMass Lowell,2.28,1.67,155.5,1.91,1.91,1.5,2.05,1.71,txRarUHC,0.438596,0.598802,0.523560,0.523560,0.037399,179.042,102.595575,0.573025,1.8,1.643168,0.912871,148.20,2.302,1.505646,0.654060,22.0,98.814,39.596925,0.400722,2.4,1.341641,0.559017,74.20,1.368,0.402828,0.294465,33.0,95,70,1.56,1.06,120.150,122.810,0.218398,0.000000,0.127881,2.32,0.464,2.758621,0.000000,0.0,0.000000,-0.35,-0.070,-9.571429,0.000000,0.0,0.000000
10551,2024-01-18,Eua Ncaa,21:00,Elon,NC Wilmington,3.66,1.30,149.5,1.91,1.91,5.5,2.10,1.67,EoP9yzrm,0.273224,0.769231,0.523560,0.523560,0.042455,264.262,241.029852,0.912087,1.2,1.643168,1.369306,259.44,3.960,4.048846,1.022436,-23.0,170.808,128.213530,0.750630,1.2,1.643168,1.369306,112.97,2.230,1.277556,0.572895,-16.0,69,79,3.76,1.43,157.206,183.608,0.672892,0.000000,0.161303,1.59,0.318,8.364780,0.000000,0.0,0.000000,-1.69,-0.338,-0.887574,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10648,2024-01-18,Eua Ncaa,23:00,California,Washington,1.90,1.90,156.5,1.83,1.83,-1.5,2.00,1.73,ITfmLpoR,0.526316,0.526316,0.546448,0.546448,0.052632,276.434,155.208369,0.561466,0.6,1.341641,2.236068,264.99,3.590,1.842254,0.513163,-32.0,157.884,120.767466,0.764913,1.2,1.643168,1.369306,101.87,1.970,1.237679,0.628264,3.0,73,61,3.63,1.67,222.396,242.256,0.000000,0.000000,0.102369,0.64,0.128,7.031250,0.622698,0.5,-0.122698,-3.39,-0.678,-1.327434,0.000000,0.0,0.000000
10649,2024-01-18,Eua Ncaa,22:30,Michigan,Illinois,2.10,1.73,153.5,1.83,1.83,1.5,1.91,1.80,GxbO4swp,0.476190,0.578035,0.546448,0.546448,0.054225,123.938,33.951029,0.273936,0.6,1.341641,2.236068,142.35,1.8

## Apostas do dia

### Back Home V3 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
10569,11:45,Libano Divisão 1,Mayrouba,Champville,2.65,Back Home
10606,07:00,Coreia Do Sul Kbl,KoGas,Suwon KT,2.79,Back Home
10608,08:35,China Cba,Guangzhou,Zhejiang Chouzhou,2.70,Back Home
10621,20:00,Brasil Nbb,Mogi,Corinthians Paulista,2.45,Back Home
